In [ ]:
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
import awkward as ak
import matplotlib.pyplot as plt
import numba
from numba import njit
import numpy as np

In [ ]:
filename ="/pnfs/psi.ch/cms/trivcat/store/mc/Run3Summer22EENanoAODv12/VBFHHto4B_CV_1_C2V_1_C3_1_TuneCP5_13p6TeV_madgraph-pythia8/NANOAODSIM/130X_mcRun3_2022_realistic_postEE_v6-v3/80000/a897cbbf-ea0b-40d5-b850-f9f24a7906a5.root"
events = NanoEventsFactory.from_root(filename, schemaclass=NanoAODSchema, entry_stop=1).events()
print("Events read:", len(events))

In [ ]:
events.GenPart=ak.with_field(events.GenPart, ak.local_index(events.GenPart, axis=1), "index")
genpart= events.GenPart
single_event=-1
if (single_event!=-1):
    genpart= events.GenPart[single_event]


isQuark = abs(genpart.pdgId) < 7
isHard = genpart.hasFlags(["fromHardProcess"])

quarks = genpart[isQuark & isHard]
quarks = quarks[quarks.genPartIdxMother!=-1]

quarks_mother = genpart[quarks.genPartIdxMother]
quarks_mother_children = quarks_mother.children
quarks_mother_children_isH = ak.sum((quarks_mother_children.pdgId == 25), axis=-1)==2
vbf_quarks = quarks[quarks_mother_children_isH]


print(quarks_mother_children_isH, quarks_mother_children_isH[0])
print(quarks.pdgId, quarks.genPartIdxMother, quarks.index)
print(vbf_quarks.pdgId, vbf_quarks.genPartIdxMother, vbf_quarks.index, genpart[vbf_quarks.genPartIdxMother].pdgId)
print(vbf_quarks.hasFlags(["isFirstCopy"]))


In [ ]:
vbf_quarks_children_idx=vbf_quarks.children.index
print(vbf_quarks.index)

print(vbf_quarks_children_idx)
print(ak.num(genpart, axis=1))
print(vbf_quarks.childrenIdxG)

print(genpart)


In [ ]:

genparts = events.GenPart

children_idxG = ak.without_parameters(genparts.childrenIdxG, behavior={})
children_idxG_flat = ak.flatten(children_idxG, axis=1)
genpart_pdgId_flat = ak.flatten(ak.without_parameters(genparts.pdgId, behavior={}), axis=1)
genpart_LastCopy_flat = ak.flatten(ak.without_parameters(genparts.hasFlags(["isLastCopy"]), behavior={}), axis=1)
genparts_flat = ak.flatten(genparts)
genpart_offsets = np.concatenate([[0],np.cumsum(ak.to_numpy(ak.num(genparts, axis=1), allow_missing=True))])
local_index_all = ak.local_index(genparts, axis=1)
local_index_vbf = ak.local_index(vbf_quarks, axis=1)

print(genpart_offsets)
print(children_idxG)
print(children_idxG_flat)
print(genpart_pdgId_flat)
print(genpart_LastCopy_flat)
print(genparts_flat.index, ak.num(genparts_flat.index, axis=0))
print(local_index_all, ak.num(local_index_all, axis=1))


In [ ]:
vbf_quark_idx = ak.to_numpy(vbf_quarks.index+genpart_offsets[:-1], allow_missing=False)
vbf_quarks_pdgId = ak.to_numpy(vbf_quarks.pdgId, allow_missing=False)
nevents=vbf_quark_idx.shape[0]
print(nevents)
print(vbf_quark_idx)

In [ ]:
@njit
def analyze_parton_from_vbf_quarks(
    vbf_quarks_idx,
    vbf_quarks_pdgId,
    children_idxG_flat,
    genpart_pdgId_flat,
    genpart_offsets,
    genpart_LastCopy_flat,
    nevents,
):
    # print input array
    print("vbf_quarks_idx", vbf_quarks_idx)
    print("vbf_quarks_pdgId", vbf_quarks_pdgId)
    print("children_idxG_flat", children_idxG_flat)
    print("genpart_pdgId_flat", genpart_pdgId_flat)
    print("genpart_offsets", genpart_offsets)
    print("genpart_LastCopy_flat", genpart_LastCopy_flat)
    print("nevents", nevents)


    # get the children ofthe vbf_quarks which have the same pdgId of the mother iteratively until we reach the last copy

    out = np.zeros(vbf_quarks_idx.shape, dtype="int64")-1

    for iev in range(vbf_quarks_idx.shape[0]):
        print("Event", iev)
        for ipart in range(vbf_quarks_idx.shape[1]):
            p_id = vbf_quarks_idx[iev][ipart]
            print("Parton", ipart)
            children_idxs = children_idxG_flat[p_id]
            print(children_idxs)
            if genpart_LastCopy_flat[p_id]:
                out[iev][ipart] = p_id
                continue
            while not genpart_LastCopy_flat[p_id]:
                for child_idx in children_idxs:
                    print("Child", child_idx)
                    print(genpart_pdgId_flat[child_idx])
                    print(vbf_quarks_pdgId[iev][ipart])
                    if genpart_pdgId_flat[child_idx] == vbf_quarks_pdgId[iev][ipart]:
                        print(genpart_LastCopy_flat[child_idx])
                        if  genpart_LastCopy_flat[child_idx]:
                            print("Found child")
                            out[iev][ipart] = child_idx
                            break
                        print(p_id, child_idx)
                        p_id = child_idx
                        print(p_id, child_idx)
                if out[iev][ipart]!=-1:
                    break


            # for child_idx in children_idxs:
            #     print("Child", child_idx)
            #     print(genpart_pdgId_flat[child_idx])
            #     print(vbf_quarks_pdgId[iev][ipart])
            #     if genpart_pdgId_flat[child_idx] == vbf_quarks_pdgId[iev][ipart]:
            #         print(genpart_LastCopy_flat[child_idx])
            #         if  genpart_LastCopy_flat[child_idx]:
            #             print("Found child")
            #             out[iev][ipart] = child_idx
            #             break
            #         children_idxs = children_idxG_flat[child_idx]


In [ ]:
analyze_parton_from_vbf_quarks(
    vbf_quark_idx,
    vbf_quarks_pdgId,
    children_idxG_flat,
    genpart_pdgId_flat,
    genpart_offsets,
    genpart_LastCopy_flat,
    nevents,
)